# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [74]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import numpy as np
import requests
import time
import json

# Import API key
from api_keys import myKey

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [75]:
#Adding to ignore the warning for appending the values in the next step.  
#Error is:  SettingWithCopyWarning:   A value is trying to be set on a copy of a slice from a DataFrame
pd.options.mode.chained_assignment = None  # default='warn'   

## Generate Cities List

In [76]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
countries = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1750)  
lngs = np.random.uniform(low=-180.000, high=180.000, size=1750)
lat_lngs = zip(lats, lngs)
#uniform is to have the numbers evenly distributed
#size is how many random numbers will be generated.
#we are saying, generate 1500 random evenly distributed numbers between -90 and 90.

# Identify nearest city for each lat, lng combination
for each in lat_lngs:
    city = citipy.nearest_city(each[0], each[1]).city_name 
    country = citipy.nearest_city(each[0], each[1]).country_code 
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        countries.append(country)

# Print the city count to confirm sufficient count
len(cities)

#612 cites are returned.   There must be at least 500 on the next part of finding the weather.  If not, then must come back to this step and increase size.

678

In [77]:
len(countries)

678

In [78]:
#??  how do I see the ln_lngs list?
print(lat_lngs)

In [79]:
print(lngs)

[-148.93157815  -77.86180803  113.13297863 ... -103.99251219 -135.25020348
   11.08375634]


In [80]:
#

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [81]:
#first, perform a check on a single city.

In [82]:
url = "http://api.openweathermap.org/data/2.5/weather?"
city = "alofi"
units = "imperial"

# Build query URL
query_url = url + "appid=" + myKey + "&q=" + city + "&units=imperial"
#note:   adding imperial units will return the temperature in farenheight and the wind speed in miles per hour
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&q=alofi&units=imperial


In [83]:
# Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()


In [84]:
print(json.dumps(weather_json, indent=4, sort_keys=True))

{
    "base": "stations",
    "clouds": {
        "all": 56
    },
    "cod": 200,
    "coord": {
        "lat": -19.06,
        "lon": -169.92
    },
    "dt": 1554073200,
    "id": 4036284,
    "main": {
        "humidity": 88,
        "pressure": 1009,
        "temp": 78.8,
        "temp_max": 78.8,
        "temp_min": 78.8
    },
    "name": "Alofi",
    "rain": {
        "3h": 0.625
    },
    "sys": {
        "country": "NU",
        "id": 7306,
        "message": 0.0043,
        "sunrise": 1554053160,
        "sunset": 1554096080,
        "type": 1
    },
    "weather": [
        {
            "description": "light rain",
            "icon": "10d",
            "id": 500,
            "main": "Rain"
        }
    ],
    "wind": {
        "deg": 250,
        "speed": 8.05
    }
}


In [85]:
#Success!   Able to retrieve a single city.
#now retrieve each individual piece of data needed for the loop

In [86]:
print(weather_json["name"])

Alofi


In [87]:
print(weather_json["coord"]["lat"])

-19.06


In [88]:
print(weather_json["main"]["temp"])

78.8


In [89]:
print(weather_json["main"]["humidity"])

88


In [90]:
print(weather_json["clouds"]["all"])

56


In [91]:
print(weather_json["wind"]["speed"])

8.05


### need a dataframe containing city and country to begin
NOTE:   the Weather API will take only a city; however, if multiples are found, API responds with a list of results that match a searching word. 
<BR>
Therefore, I will begin with a list of cities and countries which can also be passed to the 
    <BR>q city name and country code divided by comma, use ISO 3166 country codes
        
<BR> https://openweathermap.org/current

In [92]:
#Begin with the city and country list which was created in the begining.

cityCountry = pd.DataFrame(list(zip(cities, countries)),
              columns=['city','country'])

cityCountry.head()

,city,country
0,mataura,pf
1,cartagena,co
2,albany,au
3,nanortalik,gl
4,kaeo,nz


In [93]:
#confirm there are at least 500 countries
len(cityCountry)

678

In [94]:
# create a data frome with the values needed for the API calls

cityCountry2 = cityCountry
cityCountry2["location"] = (cityCountry2["city"] + ", " + cityCountry2["country"])
cityCountry2["newCity"] = ""
cityCountry2["newCountry"] = ""
cityCountry2["lat"] = ""
cityCountry2["long"] = ""
cityCountry2["tempF"] = ""
cityCountry2["humidity"] = ""
cityCountry2["cloud"] = ""
cityCountry2["windMPH"] = ""
cityCountry2["maxTemp"] = ""
cityCountry2["timeStamp"] = ""
cityCountry2.head()

,city,country,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp,timeStamp
0,mataura,pf,"mataura, pf",,,,,,,,,,
1,cartagena,co,"cartagena, co",,,,,,,,,,
2,albany,au,"albany, au",,,,,,,,,,
3,nanortalik,gl,"nanortalik, gl",,,,,,,,,,
4,kaeo,nz,"kaeo, nz",,,,,,,,,,


## Retrieve Data from the API
Practice on 10 cities before attemping to get data for 500+ cities

In [95]:
#create a new df with only 10 records
cityCountrySample = cityCountry2.head(10)
cityCountrySample.head(20)

,city,country,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp,timeStamp
0,mataura,pf,"mataura, pf",,,,,,,,,,
1,cartagena,co,"cartagena, co",,,,,,,,,,
2,albany,au,"albany, au",,,,,,,,,,
3,nanortalik,gl,"nanortalik, gl",,,,,,,,,,
4,kaeo,nz,"kaeo, nz",,,,,,,,,,
5,barrow,us,"barrow, us",,,,,,,,,,
6,tuatapere,nz,"tuatapere, nz",,,,,,,,,,
7,benha,eg,"benha, eg",,,,,,,,,,
8,vilyuysk,ru,"vilyuysk, ru",,,,,,,,,,
9,busselton,au,"busselton, au",,,,,,,,,,


In [96]:
cityCountrySample.dtypes

city          object
country       object
location      object
newCity       object
newCountry    object
lat           object
long          object
tempF         object
humidity      object
cloud         object
windMPH       object
maxTemp       object
timeStamp     object
dtype: object

In [97]:
#cast the timeStamp as a date type
cityCountrySample['timeStamp'] = pd.to_datetime(cityCountrySample['timeStamp'])

In [98]:
#ensure I can get a single response using parameters
parameters = {"appid": myKey,
             "units": "imperial",
             "q": "vaini, to"}
base_url = "http://api.openweathermap.org/data/2.5/weather?"

In [99]:
#get the API response
singleResponse = requests.get(base_url, params=parameters)

In [100]:
#confirm the URL
print(singleResponse.url)

http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=vaini%2C+to


In [101]:
#convert the response to JSON 
singleResponseJ = singleResponse.json()

In [102]:
#print the response to view the data
print(json.dumps(singleResponseJ, indent=4, sort_keys=True))

{
    "base": "stations",
    "clouds": {
        "all": 75
    },
    "cod": 200,
    "coord": {
        "lat": -21.2,
        "lon": -175.2
    },
    "dt": 1554073200,
    "id": 4032243,
    "main": {
        "humidity": 94,
        "pressure": 1012,
        "temp": 78.8,
        "temp_max": 78.8,
        "temp_min": 78.8
    },
    "name": "Vaini",
    "sys": {
        "country": "TO",
        "id": 7285,
        "message": 0.005,
        "sunrise": 1554140885,
        "sunset": 1554183652,
        "type": 1
    },
    "visibility": 10000,
    "weather": [
        {
            "description": "broken clouds",
            "icon": "04d",
            "id": 803,
            "main": "Clouds"
        }
    ],
    "wind": {
        "deg": 100,
        "speed": 12.75
    }
}


In [103]:
#success!  I can get a valid URL using parameters.  

###  Complete for 10 Records
Before applying to all records, complete the for loop for a small sample.

In [104]:
import datetime as dt
timeTest = pd.Series([dt.datetime.now()])
print(timeTest)

0   2019-03-31 19:25:15.162130
dtype: datetime64[ns]


In [105]:
# create a parameters dict that will be updated with new city each iteration
#per API, imperial units will obtain temperature in F and wind speed in mph
parameters = {"appid": myKey,
             "units": "imperial"}

print("---------------------------------------------") 
print("Beginning Data Retrieval")
print("---------------------------------------------") 
      
#begin by knowing the number of cities
print("Number of cities to obtain data: " + str(len(cityCountry))) 
print("---------------------------------------------")       

# Loop through the cityCountrySample and perform a search on each
for index, row in cityCountrySample.iterrows():
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    city = row['city']
    country = row['country']

    # update address key value
    parameters['q'] = f"{city},{country}"

    # make request
    cities_data = requests.get(base_url, params=parameters)
    
    #print the record number  ??? how do I print the index number?
    print("Retrieving index # " + str(index) + " | " + row["city"] + ", " + row["country"])
    
    #confirm the URL
    print("  " + (cities_data.url))
        
    # convert to json
    citiesJ = cities_data.json()
    
    try:
        cityCountrySample.loc[index, "newCity"] = citiesJ["name"]
    except:
        print("     Missing city name... skipping data.")   
        # ??is there a way to move to the sleep time??
    
    try:    
        cityCountrySample.loc[index, "newCountry"] = citiesJ["sys"]["country"]
    except: 
        print("     Missing country name... skipping data.")
 
    try:    
        cityCountrySample.loc[index, "lat"] = citiesJ["coord"]["lat"]
    except: 
        print("     Missing latitude... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "long"] = citiesJ["coord"]["lon"]
    except: 
        print("     Missing longitude... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "tempF"] = citiesJ["main"]["temp"]
    except: 
        print("     Missing temp(F)... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "humidity"] = citiesJ["main"]["humidity"]
    except: 
        print("     Missing humidity... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "cloud"] = citiesJ["clouds"]["all"]
    except: 
        print("     Missing %of cloudiness... skipping data.")
     
    try:    
        cityCountrySample.loc[index, "windMPH"] = citiesJ["wind"]["speed"]
    except: 
        print("     Missing wind speed... skipping data.")
        
    try:    
        cityCountrySample.loc[index, "maxTemp"] = citiesJ["main"]["temp_max"]
    except: 
        print("     Missing maximum temperature... skipping data.")
    
    #add the timestamp when the record was added.   ??does not work.
    try:
        cityCountrySample.loc[index, "timeStamp"] = pd.Series([dt.datetime.now()])
    except:
        dumbVariable = 1
    
    #the openweather API is free for up to 60 calls per minute.  Add a sleep timer to wait .75 seconds between calls.
    time.sleep(.75)

   

#At least 500 cities must be included for the next step of creating graphics.   Ensure there are at least 500 cities.

#obtain the number of records where a city was not found
noRecord = len(cityCountrySample[cityCountrySample['newCity'] == ""])
print("Records not found in the weather database: " + str(noRecord))

#subtract these from the number of records at the beginning
endRecords = (len(cityCountry)) - noRecord

initRecords = (len(cityCountry))

print("---------------------------------------------")
print("Beginning Records: " + str(initRecords))
print("Ending Records: " + str(endRecords))

if endRecords < 500:
    print("The data sample is less than 500, which is too small.  Please start from the beginning to obtain a new data sample.")
    print("---------------------------------------------")   
    #store the dataframe to a csv so the user can get details if needed.
    cityCountrySample.to_csv("weatherData_sample10.csv", encoding='utf-8', index=False)
    print("All data collected is stored in the same directory as this python file.   Filename = weatherData_sample10.csv")
    print("Date Retrieval Complete")
    print("---------------------------------------------")   

else: 
    print("There are more than 500 ending records!   Continue to the graphs.")
    cityCountrySample.to_csv("weatherData_sample10.csv", encoding='utf-8', index=False)
    print("All data collected is stored in the same directory as this python file.   Filename = weatherData_sample10.csv")

print("---------------------------------------------")   
print("Date Retrieval Complete")
print("---------------------------------------------")   


# Visualize to confirm new city and new country appear
cityCountrySample.head(10)


---------------------------------------------
Beginning Data Retrieval
---------------------------------------------
Number of cities to obtain data: 678
---------------------------------------------
Retrieving index # 0 | mataura, pf
  http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=mataura%2Cpf
     Missing city name... skipping data.
     Missing country name... skipping data.
     Missing latitude... skipping data.
     Missing longitude... skipping data.
     Missing temp(F)... skipping data.
     Missing humidity... skipping data.
     Missing %of cloudiness... skipping data.
     Missing wind speed... skipping data.
     Missing maximum temperature... skipping data.
Retrieving index # 1 | cartagena, co
  http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&q=cartagena%2Cco
Retrieving index # 2 | albany, au
  http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb

,city,country,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp,timeStamp
0,mataura,pf,"mataura, pf",,,,,,,,,,NaT
1,cartagena,co,"cartagena, co",Cartagena,CO,5.03,-75.64,70.34,100,40,4.7,71.6,NaT
2,albany,au,"albany, au",Albany,AU,-35.02,117.88,62.6,72,64,9.17,62.6,NaT
3,nanortalik,gl,"nanortalik, gl",Nanortalik,GL,60.14,-45.24,26.9,100,20,22.53,26.9,NaT
4,kaeo,nz,"kaeo, nz",Kaeo,NZ,-35.1,173.78,70,96,92,14,70,NaT
5,barrow,us,"barrow, us",Barrow,US,39.51,-90.4,41.58,48,1,6.93,43,NaT
6,tuatapere,nz,"tuatapere, nz",Tuatapere,NZ,-46.13,167.69,57,64,44,5.01,57,NaT
7,benha,eg,"benha, eg",,,,,,,,,,NaT
8,vilyuysk,ru,"vilyuysk, ru",Vilyuysk,RU,63.75,121.63,20.24,77,20,3.06,20.24,NaT
9,busselton,au,"busselton, au",Busselton,AU,-33.64,115.35,68.04,49,80,12.01,70,NaT


In [106]:
#remove the rows with no newCity  These are the records which were not found 
cityCountrySample2 = cityCountrySample[cityCountrySample.newCity != ""].reset_index()
cityCountrySample2.head(10)

,index,city,country,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp,timeStamp
0,1,cartagena,co,"cartagena, co",Cartagena,CO,5.03,-75.64,70.34,100,40,4.7,71.6,NaT
1,2,albany,au,"albany, au",Albany,AU,-35.02,117.88,62.6,72,64,9.17,62.6,NaT
2,3,nanortalik,gl,"nanortalik, gl",Nanortalik,GL,60.14,-45.24,26.9,100,20,22.53,26.9,NaT
3,4,kaeo,nz,"kaeo, nz",Kaeo,NZ,-35.1,173.78,70,96,92,14,70,NaT
4,5,barrow,us,"barrow, us",Barrow,US,39.51,-90.4,41.58,48,1,6.93,43,NaT
5,6,tuatapere,nz,"tuatapere, nz",Tuatapere,NZ,-46.13,167.69,57,64,44,5.01,57,NaT
6,8,vilyuysk,ru,"vilyuysk, ru",Vilyuysk,RU,63.75,121.63,20.24,77,20,3.06,20.24,NaT
7,9,busselton,au,"busselton, au",Busselton,AU,-33.64,115.35,68.04,49,80,12.01,70,NaT


## Retrieve data on all cities 
Since the practice run on 10 entries works well, complete the exercise on all records.

In [107]:
#create a new df to store the values from the API
apiData = cityCountry2
apiData.head(20)

,city,country,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp,timeStamp
0,mataura,pf,"mataura, pf",,,,,,,,,,
1,cartagena,co,"cartagena, co",,,,,,,,,,
2,albany,au,"albany, au",,,,,,,,,,
3,nanortalik,gl,"nanortalik, gl",,,,,,,,,,
4,kaeo,nz,"kaeo, nz",,,,,,,,,,
5,barrow,us,"barrow, us",,,,,,,,,,
6,tuatapere,nz,"tuatapere, nz",,,,,,,,,,
7,benha,eg,"benha, eg",,,,,,,,,,
8,vilyuysk,ru,"vilyuysk, ru",,,,,,,,,,
9,busselton,au,"busselton, au",,,,,,,,,,


In [ ]:
# create a parameters dict that will be updated with new city each iteration
#per API, imperial units will obtain temperature in F and wind speed in mph
parameters = {"appid": myKey,
             "units": "imperial"}

print("---------------------------------------------") 
print("Beginning Data Retrieval")
print("---------------------------------------------") 
      
#begin by knowing the number of cities
print("Number of cities to obtain data: " + str(len(cityCountry))) 
print("---------------------------------------------")       

# Loop through the cityCountrySample and perform a search on each
for index, row in apiData.iterrows():
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    city = row['city']
    country = row['country']

    # update address key value
    parameters['q'] = f"{city},{country}"

    # make request
    cities_data = requests.get(base_url, params=parameters)
    
    #print the record number  ??? how do I print the index number?
    print("Retrieving index # " + str(index) + " | " + row["city"] + ", " + row["country"])
    
    #confirm the URL
    print("  " + (cities_data.url))
        
    # convert to json
    citiesJ = cities_data.json()
    
    try:
        apiData.loc[index, "newCity"] = citiesJ["name"]
    except:
        print("     Missing city name... skipping data.")     
    
    try:    
        apiData.loc[index, "newCountry"] = citiesJ["sys"]["country"]
    except: 
        print("     Missing country name... skipping data.")
 
    try:    
        apiData.loc[index, "lat"] = citiesJ["coord"]["lat"]
    except: 
        print("     Missing latitude... skipping data.")
        
    try:    
        apiData.loc[index, "long"] = citiesJ["coord"]["lon"]
    except: 
        print("     Missing longitude... skipping data.")
        
    try:    
        apiData.loc[index, "tempF"] = citiesJ["main"]["temp"]
    except: 
        print("     Missing temp(F)... skipping data.")
        
    try:    
        apiData.loc[index, "humidity"] = citiesJ["main"]["humidity"]
    except: 
        print("     Missing humidity... skipping data.")
        
    try:    
        apiData.loc[index, "cloud"] = citiesJ["clouds"]["all"]
    except: 
        print("     Missing %of cloudiness... skipping data.")
     
    try:    
        apiData.loc[index, "windMPH"] = citiesJ["wind"]["speed"]
    except: 
        print("     Missing wind speed... skipping data.")
        
    try:    
        apiData.loc[index, "maxTemp"] = citiesJ["main"]["temp_max"]
    except: 
        print("     Missing maximum temperature... skipping data.")
    
    try:
        #add the timestamp when the record was retrieved
        apiData.loc[index, "timeStamp"] = pd.Series([dt.datetime.now()])
    except:
        dumbVariable = 1
    
    #the openweather API is free for up to 60 calls per minute.  Add a sleep timer to wait .75 seconds between calls.
    time.sleep(.75)

   
#At least 500 cities must be included for the next step of creating graphics.   Ensure there are at least 500 cities.

#obtain the number of records where a city was not found
noRecord = len(apiData[apiData['newCity'] == ""])
print("Records not found in the weather database: " + str(noRecord))

#subtract these from the number of records at the beginning
endRecords = (len(cityCountry)) - noRecord

initRecords = (len(cityCountry))

print("---------------------------------------------")
print("Beginning # of Cities: " + str(initRecords))
print("Ending # of Cities: " + str(endRecords))
print("Cities not found in the weather database: " + str(noRecord))

if endRecords < 500:
    print("The data sample is less than 500, which is too small.  Please start from the beginning to obtain a new data sample.")
    print("---------------------------------------------")  
    
    #save the data to a csv in case the user needs to view the details
    apiData.to_csv("weatherData_TooSmall.csv", encoding='utf-8', index=False)
    print("All data collected is stored in the same directory as this python file.   Filename = weatherData_TooSmall.csv")
    
    print("Date Retrieval Complete")
    print("---------------------------------------------")   

else: 
    print("There are more than 500 ending records!   Continue to the graphs.")
    apiData.to_csv("weatherData.csv", encoding='utf-8', index=False)
    print("All data collected is stored in the same directory as this python file.   Filename = weatherData.csv")
    


print("---------------------------------------------")   
print("Date Retrieval Complete")
print("---------------------------------------------")   
    
# Visualize to confirm new city and new country appear
apiData.head(10)


In [109]:
#remove the rows with no newCity  These are the records which were not found 
apiData2 = apiData[apiData.newCity != ""].reset_index()
apiData2.head(10)

,index,city,country,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp,timeStamp
0,1,cartagena,co,"cartagena, co",Cartagena,CO,5.03,-75.64,70.34,100,40,4.7,71.6,
1,2,albany,au,"albany, au",Albany,AU,-35.02,117.88,62.6,72,64,9.17,62.6,
2,3,nanortalik,gl,"nanortalik, gl",Nanortalik,GL,60.14,-45.24,26.9,100,20,22.53,26.9,
3,4,kaeo,nz,"kaeo, nz",Kaeo,NZ,-35.1,173.78,70,96,92,14,70,
4,5,barrow,us,"barrow, us",Barrow,US,39.51,-90.4,41.58,48,1,6.93,43,
5,6,tuatapere,nz,"tuatapere, nz",Tuatapere,NZ,-46.13,167.69,57,64,44,5.01,57,
6,8,vilyuysk,ru,"vilyuysk, ru",Vilyuysk,RU,63.75,121.63,20.24,77,20,3.06,20.24,
7,9,busselton,au,"busselton, au",Busselton,AU,-33.64,115.35,68.04,49,80,12.01,70,
8,10,rikitea,pf,"rikitea, pf",Rikitea,PF,-23.12,-134.97,76.76,100,24,18.5,76.76,
9,12,uyuni,bo,"uyuni, bo",Uyuni,BO,-20.46,-66.82,46.61,92,88,2.39,46.61,


In [110]:
#confirm the number matches the end processing summary offered above
apiData2.count()

index         590
city          590
country       590
location      590
newCity       590
newCountry    590
lat           590
long          590
tempF         590
humidity      590
cloud         590
windMPH       590
maxTemp       590
timeStamp     590
dtype: int64

### Convert the datatypes for matplotlib

In [111]:
#copy the df.  This is not needed, but completing this step in case I need to refer back to apiData2
apiData3 = apiData2
apiData3.dtypes

index          int64
city          object
country       object
location      object
newCity       object
newCountry    object
lat           object
long          object
tempF         object
humidity      object
cloud         object
windMPH       object
maxTemp       object
timeStamp     object
dtype: object

In [114]:
#change the data types so numbers are floats, and also the datetime
apiData3.lat= apiData3.lat.astype(float)
apiData3.long= apiData3.lat.astype(float)
apiData3.tempF= apiData3.lat.astype(float)
apiData3.humidity= apiData3.lat.astype(float)
apiData3.cloud= apiData3.lat.astype(float)
apiData3.windMPH= apiData3.lat.astype(float)
apiData3.maxTemp= apiData3.lat.astype(float)
apiData3['timeStamp'] = pd.to_datetime(apiData3['timeStamp'])
apiData3.dtypes

index                  int64
city                  object
country               object
location              object
newCity               object
newCountry            object
lat                  float64
long                 float64
tempF                float64
humidity             float64
cloud                float64
windMPH              float64
maxTemp              float64
timeStamp     datetime64[ns]
dtype: object

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot